In [14]:
import psycopg2
import undetected_chromedriver as uc
uc.install()
from selectorlib import Extractor
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np

In [28]:
# Establish a connection to the database by creating a cursor object
# The PostgreSQL server must be accessed through the PostgreSQL APP or Terminal Shell

# conn = psycopg2.connect("dbname=suppliers port=5432 user=postgres password=postgres")

# Or:
conn = psycopg2.connect(host="localhost", port = 5432, database="addresses", user="postgres", password="password")

# Create a cursor object
cur = conn.cursor()

# A sample query of all data from the "vendors" table in the "suppliers" database
cur.execute("""SELECT * FROM house WHERE post_comm ilike('hackettstown')""")
query_results = cur.fetchall()
print(query_results)

# Close the cursor and connection to so the server can allocate
# bandwidth to other requests
cur.close()
conn.close()

[('NJ', 'HACKETTSTOWN', 'OSPREY', '', '83', 'NULL', '', '07840', '83  OSPREY NULL', 908311), ('NJ', 'HACKETTSTOWN', 'WOOD DUCK', '', '70', 'COURT', '', '07840', '70  WOOD DUCK COURT', 908312), ('NJ', 'HACKETTSTOWN', 'WOOD DUCK', '', '78', 'COURT', '', '07840', '78  WOOD DUCK COURT', 908387), ('NJ', 'HACKETTSTOWN', 'WOOD DUCK', '', '10', 'COURT', '', '07840', '10  WOOD DUCK COURT', 908388), ('NJ', 'HACKETTSTOWN', 'GOLDFINCH', '', '62', 'DRIVE', '', '07840', '62  GOLDFINCH DRIVE', 908313), ('NJ', 'HACKETTSTOWN', 'WOOD DUCK', '', '11', 'COURT', '', '07840', '11  WOOD DUCK COURT', 908389), ('NJ', 'HACKETTSTOWN', 'WOOD DUCK', '', '22', 'COURT', '', '07840', '22  WOOD DUCK COURT', 908390), ('NJ', 'HACKETTSTOWN', 'RAINIER', '', '12', 'COURT', '', '07840', '12  RAINIER COURT', 908391), ('NJ', 'HACKETTSTOWN', 'MOURNING DOVE', '', '9', 'COURT', '', '07840', '9  MOURNING DOVE COURT', 908392), ('NJ', 'HACKETTSTOWN', 'MEADOWLARK', '', '9', 'LANE', '', '07840', '9  MEADOWLARK LANE', 908270), ('NJ', 

In [29]:
query_results[1]

('NJ',
 'HACKETTSTOWN',
 'WOOD DUCK',
 '',
 '70',
 'COURT',
 '',
 '07840',
 '70  WOOD DUCK COURT',
 908312)

In [31]:
town = query_results[1][1]
street = query_results[1][8]
zipcode = query_results[1][7]
state = query_results[1][0]

In [36]:
test = street + ' ' + town + ', ' + state + ' ' + zipcode

# Scrape resident names.

In [38]:
ChromeOptions = webdriver.ChromeOptions()
ChromeOptions.add_argument('--incognito')
# ChromeOptions.add_argument('--headless')
driver = webdriver.Chrome(options=ChromeOptions)
# driver = uc.Chrome()
driver.get("https://thatsthem.com/reverse-address-lookup")

In [42]:
driver.find_element_by_id('address').send_keys(test.title())

In [43]:
driver.find_element_by_css_selector('#app > div > main > section.ThatsThem-banner > div > form > div > button').click()

In [27]:
driver.find_element_by_css_selector('#app > form > div > div > div.Modal-footer > div > div > button.Button.Button--red').click()

In [45]:
raw_html = driver.page_source

In [47]:
resident_info = Extractor.from_yaml_string("""
    main_card:
        css: 'div.ThatsThem-record:nth-of-type(n+2) div.ThatsThem-well'
        xpath: null
        multiple: true
        type: Text
        children:
            address:
                css: div.ThatsThem-record-address
                xpath: null
                multiple: true
                type: Text
            phone_number:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(1)'
                xpath: null
                multiple: true
                type: Text
            alt_phone:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(2)'
                xpath: null
                multiple: true
                type: Text
            email:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(3)'
                xpath: null
                multiple: true
                type: Text
            length_of_residence:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(4)'
                xpath: null
                type: Text
            household_size:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(5)'
                xpath: null
                multiple: true
                type: Text
            ip_address:
                css: 'dd.col-md-8:nth-of-type(6)'
                xpath: null
                multiple: true
                type: Text
            net_worth:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(1)'
                xpath: null
                multiple: true
                type: Text
            estimated_income:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(2)'
                xpath: null
                multiple: true
                type: Text
            education:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(3)'
                xpath: null
                multiple: true
                type: Text
            occupation:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(4)'
                xpath: null
                multiple: true
                type: Text
            language:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(5)'
                xpath: null
                multiple: true
                type: Text
            name:
                css: 'h2 span'
                xpath: null
                multiple: true
                type: Text
""")
raw_data = resident_info.extract(raw_html)

In [53]:
hey = pd.DataFrame(raw_data['main_card'])

In [69]:
hey

,address,phone_number,alt_phone,email,length_of_residence,household_size,ip_address,net_worth,estimated_income,education,occupation,language,name
0,"70 Wood Duck , CT Hackettstown New Jersey 07840",,[],[Scoobydoo2249@aol.com],,[],[],[],[],[],[],[],[Kayla M Law]
1,"70 Wood Duck , CT Hackettstown New Jersey 07840",,[],[Laurenc81@optimum.net],,[],[174.252.69.41],[],[],[],[],[],[Lauren A Carolan]
2,"70 Wood Duck , CT Hackettstown New Jersey 07840",908-448-1907,[],[Pthomas@onebox.com],,[],[63.150.131.223],[],[],[],[],[],[Patricia Thomas]


In [65]:
hey['address'] = hey['address'].str.get(0)

In [68]:
hey['phone_number'] = hey['phone_number'].str.get(0)

In [37]:
driver.close()